In [1]:
import gurobipy as gp
import numpy as np

In [9]:
P_ABC = np.array([[[0.254, 0.24 ],
        [0.   , 0.   ]],

       [[0.   , 0.   ],
        [0.259, 0.247]]])

In [3]:
def func(b, a1, a2):
    P_A = np.array([sum([P_ABC[a,b,c] for b,c in np.ndindex(2,2)]) for a in range(2)])
    P_B = np.array([sum([P_ABC[a,b,c] for a,c in np.ndindex(2,2)]) for b in range(2)])
    P_C = np.array([sum([P_ABC[a,b,c] for a,b in np.ndindex(2,2)]) for c in range(2)])


    m = gp.Model()
    m.setParam('OutputFlag', 0)
    Q_Ab_BC_giv_As = m.addMVar((2, 2, 2, 2), name="Q", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1) # A interupted
    R_A1A2_B1B2_C1C2_giv_As = m.addMVar((2, 2, 2, 2, 2, 2, 2), name="R", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1) # inflation
    m.update()


    # consistency
    for a, b, c in np.ndindex(2,2,2):
        m.addConstr(P_ABC[a,b,c] == Q_Ab_BC_giv_As[a,b,c,a])


    ## normalization
    for a_s in np.ndindex(2):
        m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,:,:,:,:,:,a_s].sum() == 1)
        m.addConstr(Q_Ab_BC_giv_As[:,:,:,a_s].sum() == 1)

    ## do conditional on P is a normal conditional on Q
    def P_B_do_A(b,a):
        return Q_Ab_BC_giv_As[:,b,:,a].sum()


    ## graph to graph relations:

    # R(A1,B1,C1|A#) = Q(Ab,B,C|A#)
    for a,b,c,a_s in np.ndindex(2,2,2,2):
        m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a,:,b,:,c,:,a_s].sum() == Q_Ab_BC_giv_As[a,b,c,a_s])

    # R(A2,B2,C2|A#) = P(A)P(C)Q(B|A#) = Q(A)Q(C)Q(B|A#)
    for a,b,c,a_s in np.ndindex(2,2,2,2):
        m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,a,:,b,:,c,a_s].sum() == P_A[a]*P_C[c]*Q_Ab_BC_giv_As[:,b,:,a_s].sum())
        # also == Q_Ab_BC_giv_As[a,b,c,a_s]?

    # R(A1,B2|A#) = Q(Ab,B|A#)
    for a_b, a_s, b in np.ndindex(2,2,2):
        m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a_b,:,:,b,:,:,a_s].sum() == Q_Ab_BC_giv_As[a_b,b,:,a_s].sum())

    # # R(C2,A2|A#) = P(A)P(C)
    # for c, a, a_s in np.ndindex(2,2,2):
    #     m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,a,:,:,:,c,a_s].sum() == P_A[a]*P_C[c])

    # R(A1,A2,C1,C2|A# = 0) = R(A1,A2,C1,C2|A# = 1)
    for a1,a2,c1,c2 in np.ndindex(2,2,2,2):
        m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a1,a2,:,:,c1,c2,0].sum() == R_A1A2_B1B2_C1C2_giv_As[a1,a2,:,:,c1,c2,1].sum())


    # independence on Q

    # (Ab ⫫ As| B) on Q: Q(Ab|B,A#=0) = Q(Ab|B, A#=1)
    for a_b in range(2):
        m.addConstr(Q_Ab_BC_giv_As[a_b,:,:,0].sum() == Q_Ab_BC_giv_As[a_b,:,:,1].sum())

    # (C ⫫ As| B) on Q: Q(C|B,A#=0) = Q(C|B,A#=1)
    for c in range(2):
        m.addConstr(Q_Ab_BC_giv_As[:,:,c,0].sum() == Q_Ab_BC_giv_As[:,:,c,1].sum())



    t = m.addVar(name="t", vtype=gp.GRB.CONTINUOUS, lb=0, ub=10)
    m.addConstr(P_B_do_A(b,a1) - P_B_do_A(b,a2) <= t)
    m.addConstr(P_B_do_A(b, a2) - P_B_do_A(b, a1) <= t)
    m.update()


    m.setObjective(t, gp.GRB.MINIMIZE)
    m.optimize()

    if m.status == gp.GRB.OPTIMAL:
        print(f"t={t.X}\nP_B_do_A({b},{a1})={Q_Ab_BC_giv_As[:,b,:,a1].X.sum()}\nP_B_do_A({b},{a2})={Q_Ab_BC_giv_As[:,b,:,a2].X.sum()}")
        print(f"|P_B_do_A({b},{a1}) - P_B_do_A({b},{a2})| =",abs(Q_Ab_BC_giv_As[:,b,:,a1].X.sum() - Q_Ab_BC_giv_As[:,b,:,a2].X.sum()))
    else:
        print("No solution found")

In [11]:
func(1,1,0)

t=0.0
P_B_do_A(1,1)=0.678533254395388
P_B_do_A(1,1)=0.678533254395388
|P_B_do_A(1,1) - P_B_do_A(1,1)| = 0.0
